Embedding Process in 3D Object RiggedHand

In [1]:
#!pip install pycryptodome
#!pip install trimesh
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib
import secrets
import struct
import trimesh
import scipy.spatial as spatial

class SecureFaceSteganography:
    def __init__(self):
        # ... (other code)
        self.LANDMARK_REGIONS = {
            'forehead': [(0.2, 0.8, 0.5), (0.8, 0.8, 0.5)],  # Wider bounding box
            'cheeks': [(0.1, 0.5, 0.5), (0.9, 0.5, 0.5)],  # Wider bounding box
            'jaw': [(0.2, 0.2, 0.5), (0.8, 0.2, 0.5)],  # Wider bounding box
            'nose': [(0.4, 0.6, 0.5), (0.6, 0.6, 0.5)],  # Added nose region
            'chin': [(0.5, 0.1, 0.5)]  #chin region
        }
        # Define SALT_SIZE and BLOCK_SIZE in the __init__ method
        self.SALT_SIZE = 16  # 128 bits for salt
        self.BLOCK_SIZE = 16 # 128 bits for block size
        self.distributed_landmarks = None  # Initialize distributed_landmarks

    def load_face_model(self, file_path):
        """Load 3D face model and prepare it for steganography"""
        try:
            loaded_data = trimesh.load(file_path)
            # Check if loaded_data is a Scene and extract the first mesh if it is
            if isinstance(loaded_data, trimesh.Scene):
                mesh = next(iter(loaded_data.geometry.values())) # Get the first mesh from the scene
            else:
                mesh = loaded_data

            return {
                'vertices': np.array(mesh.vertices),
                'faces': np.array(mesh.faces),
                'normals': np.array(mesh.vertex_normals)
            }
        except Exception as e:
            raise ValueError(f"Error loading face model: {str(e)}")

    def identify_landmark_vertices(self, vertices, landmark_regions=None):
        if landmark_regions is None:
            landmark_regions = self.LANDMARK_REGIONS

        # Use stored landmarks if available, otherwise generate
        if self.distributed_landmarks is None:
            num_landmarks = min(1000, len(vertices))
            self.distributed_landmarks = np.random.choice(len(vertices), num_landmarks, replace=False)

        return sorted(self.distributed_landmarks)  # Return stored landmarks

    def compute_vertex_importance(self, vertices, faces, normals):
        """Compute vertex importance based on geometric features"""
        importance = np.zeros(len(vertices))

        # Calculate curvature (simplified)
        for i, vertex in enumerate(vertices):
            # Find connected vertices
            connected = faces[np.any(faces == i, axis=1)]
            connected = np.unique(connected.flatten())
            connected = connected[connected != i]

            # Calculate local curvature
            if len(connected) > 0:
                connected_vertices = vertices[connected]
                mean_position = np.mean(connected_vertices, axis=0)
                displacement = np.linalg.norm(vertex - mean_position)
                importance[i] = displacement

        return importance

    def generate_passphrase(self, length=32):
        """Generate a cryptographically secure random passphrase"""
        charset = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&*"
        return ''.join(secrets.choice(charset) for _ in range(length))

    def derive_key(self, passphrase, salt):
        """Derive encryption key using SHA-256 with salt"""
        key_material = hashlib.sha256()
        key_material.update(passphrase.encode('utf-8'))
        key_material.update(salt)
        return key_material.digest()

    def encrypt_message(self, message, passphrase):
        """Encrypt a message using AES-128-CBC with secure key derivation"""
        salt = secrets.token_bytes(self.SALT_SIZE)
        key = self.derive_key(passphrase, salt)
        iv = secrets.token_bytes(self.BLOCK_SIZE)

        cipher = AES.new(key, AES.MODE_CBC, iv)
        padded_data = pad(message.encode('utf-8'), self.BLOCK_SIZE)
        ciphertext = cipher.encrypt(padded_data)

        return salt + iv + ciphertext

    def decrypt_message(self, encrypted_data, passphrase):
        """Decrypt a message using AES-128-CBC"""
        salt = encrypted_data[:self.SALT_SIZE]
        iv = encrypted_data[self.SALT_SIZE:self.SALT_SIZE + self.BLOCK_SIZE]
        ciphertext = encrypted_data[self.SALT_SIZE + self.BLOCK_SIZE:]

        key = self.derive_key(passphrase, salt)
        cipher = AES.new(key, AES.MODE_CBC, iv)

        # Attempt to decrypt. Handle potential padding errors
        try:
            padded_message = cipher.decrypt(ciphertext)
            return unpad(padded_message, self.BLOCK_SIZE).decode('utf-8')
        except ValueError:
            print("Warning: Decryption failed. Possible data corruption or incorrect passphrase.")
            return None  # or raise a more specific exception


    def embed_in_face_model(self, face_model, encrypted_data):
        """Embed encrypted data in 3D face model using geometric features"""
        vertices = face_model['vertices'].copy()
        faces = face_model['faces']
        normals = face_model['normals']

        # Get landmarks and importance metrics
        landmark_vertices = self.identify_landmark_vertices(vertices)
        importance = self.compute_vertex_importance(vertices, faces, normals)

        # Convert encrypted data to bit array
        bit_array = ''.join(format(byte, '08b') for byte in encrypted_data)

        # Check if the message is too large
        if len(bit_array) > len(landmark_vertices):
            raise ValueError(f"Message too large for this face model. Message bits: {len(bit_array)}, Available vertices: {len(landmark_vertices)}")

        # Embed data in vertices near landmarks using rounding and a larger scale factor
        for i, bit in enumerate(bit_array):
            vertex_idx = landmark_vertices[i]
            scale_factor = 0.001 * (1 - importance[vertex_idx])  # Significantly increased scale_factor

            # Apply rounding to the modification
            modification = round(float(bit) * scale_factor, 6)  # Round to 6 decimal places

            vertices[vertex_idx] += modification * normals[vertex_idx]

        face_model['vertices'] = vertices
        return face_model

    def extract_from_face_model(self, face_model, original_model, message_length):
        """Extract encrypted data from modified 3D face model"""
        modified_vertices = face_model['vertices']
        original_vertices = original_model['vertices']
        landmark_vertices = self.identify_landmark_vertices(original_vertices)

        bits = []
        for i in range(message_length * 8):  # Iterate over the expected number of bits
            if i >= len(landmark_vertices):
                print(f"Warning: Not enough landmark vertices to extract the full message. Extracted {i} bits, expected {message_length * 8} bits.")
                break  # Stop if we run out of landmark vertices
            vertex_idx = landmark_vertices[i]

            # Compare modified vertex with original
            diff = modified_vertices[vertex_idx] - original_vertices[vertex_idx]
            # Extract bit based on vertex displacement using a tolerance

            extracted_bit = np.dot(diff, original_model['normals'][vertex_idx]) #Using original normals

            #Using tolerance instead of direct comparison with 0
            bit = '1' if extracted_bit > 1e-6 else '0'
            bits.append(bit)

        # Convert bits back to bytes
        extracted_data = bytes(
            int(''.join(bits[i:i + 8]), 2) for i in range(0, len(bits), 8)
        )

        return extracted_data


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 26.4 MB/s eta 0:00:00


In [2]:
def main():
    # Initialize steganography system
    stego = SecureFaceSteganography()

    # Load 3D face model (example path)
    face_model = stego.load_face_model('/content/drive/MyDrive/OutputPaper2/RiggedHand.obj')
    original_model = {key: value.copy() for key, value in face_model.items()}

    # Generate secure passphrase
    passphrase = stego.generate_passphrase()
    print(f"Generated passphrase: {passphrase}")

    # Message to hide - Reduced length to fit within the model's capacity
    secret_message = "short"

    # Encrypt the message
    encrypted_data = stego.encrypt_message(secret_message, passphrase)

    # Embed encrypted data in face model
    modified_model = stego.embed_in_face_model(face_model, encrypted_data)

    # Extract and decrypt
    extracted_data = stego.extract_from_face_model(
        modified_model, original_model, len(encrypted_data)
    )
    decrypted_message = stego.decrypt_message(extracted_data, passphrase)

    print(f"Original message: {secret_message}")
    print(f"Decrypted message: {decrypted_message}")

    # Save modified model
    mesh = trimesh.Trimesh(
        vertices=modified_model['vertices'],
        faces=modified_model['faces']
    )
    mesh.export('/content/drive/MyDrive/OutputPaper2/modified_RiggedHand.obj')

if __name__ == "__main__":
    main()

Generated passphrase: An9X*!p9vvO8IF!25Pe8o8wUuaq8hqlz
Original message: short
Decrypted message: short


Evaluation RiggedHand

In [ ]:
#!pip install 'pyglet<2'
#!apt-get install -y libglu1-mesa-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libglu1-mesa-dev is already the newest version (9.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
#!pip install pyglet
#!pip install pyglet<2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import trimesh
from scipy.stats import wasserstein_distance
from skimage.metrics import structural_similarity
import math
from scipy.spatial import KDTree
import os

class Stego3DEvaluator:
    def __init__(self):
        """Initialize the 3D steganography evaluator"""
        self.metrics = {}

    def preprocess_models(self, original_mesh, stego_mesh):
        """Preprocess and align models to ensure compatibility"""
        # Center both meshes
        original_mesh.vertices -= original_mesh.vertices.mean(axis=0)
        stego_mesh.vertices -= stego_mesh.vertices.mean(axis=0)

        # Scale to unit cube
        original_scale = np.max(np.abs(original_mesh.vertices))
        stego_scale = np.max(np.abs(stego_mesh.vertices))

        original_mesh.vertices /= original_scale
        stego_mesh.vertices /= stego_scale

        return original_mesh, stego_mesh

    def align_vertices(self, original_vertices, stego_vertices):
        """Align vertices between models using nearest neighbor matching"""
        # Build KD-tree for faster nearest neighbor search
        tree = KDTree(original_vertices)

        # Find nearest neighbors for each stego vertex
        distances, indices = tree.query(stego_vertices)

        # Create aligned vertex arrays
        aligned_original = original_vertices[indices]
        aligned_stego = stego_vertices

        return aligned_original, aligned_stego, distances

    def load_models(self, original_path, stego_path, align=True):
        """Load and preprocess original and stego 3D models"""
        try:
            # Load meshes
            original_mesh = trimesh.load(original_path)
            stego_mesh = trimesh.load(stego_path)

            # Check if loaded data is a Scene and extract the first mesh if it is
            if isinstance(original_mesh, trimesh.Scene):
                original_mesh = next(iter(original_mesh.geometry.values())) # Get the first mesh from the scene
            if isinstance(stego_mesh, trimesh.Scene):
                stego_mesh = next(iter(stego_mesh.geometry.values())) # Get the first mesh from the scene

            print(f"Original vertices: {len(original_mesh.vertices)}")
            print(f"Stego vertices: {len(stego_mesh.vertices)}")

            # Preprocess meshes
            original_mesh, stego_mesh = self.preprocess_models(original_mesh, stego_mesh)

            if align:
                # Align vertices
                self.original_vertices, self.stego_vertices, distances = self.align_vertices(
                    original_mesh.vertices,
                    stego_mesh.vertices
                )

                # Print alignment statistics
                print(f"\nAlignment Statistics:")
                print(f"Mean distance: {np.mean(distances):.6f}")
                print(f"Max distance: {np.max(distances):.6f}")
                print(f"Aligned vertices: {len(self.original_vertices)}")
            else:
                if len(original_mesh.vertices) != len(stego_mesh.vertices):
                    raise ValueError("Models have different number of vertices and alignment is disabled")
                self.original_vertices = original_mesh.vertices
                self.stego_vertices = stego_mesh.vertices

            self.original_mesh = original_mesh
            self.stego_mesh = stego_mesh

            return True

        except Exception as e:
            print(f"Error loading models: {str(e)}")
            return False

    def calculate_vertex_error_map(self):
        """Calculate and visualize vertex-wise errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)
        return errors



    def calculate_psnr(self):
        """Calculate Peak Signal-to-Noise Ratio"""
        mse = np.mean((self.original_vertices - self.stego_vertices) ** 2)
        if mse == 0:
            return float('inf')

        max_val = np.max(self.original_vertices) - np.min(self.original_vertices)
        psnr = 20 * math.log10(max_val / math.sqrt(mse))
        self.metrics['PSNR'] = psnr
        return psnr

    def calculate_ssim(self):
        """Calculate Structural Similarity Index"""
        orig_reshaped = self.original_vertices.reshape(-1, 3)
        stego_reshaped = self.stego_vertices.reshape(-1, 3)

        ssim_scores = []
        for i in range(3):
            score = structural_similarity(
                orig_reshaped[:, i],
                stego_reshaped[:, i],
                data_range=np.max(orig_reshaped[:, i]) - np.min(orig_reshaped[:, i])
            )
            ssim_scores.append(score)

        ssim = np.mean(ssim_scores)
        self.metrics['SSIM'] = ssim
        return ssim

    def calculate_mse(self):
        """Calculate Mean Squared Error"""
        mse = mean_squared_error(self.original_vertices, self.stego_vertices)
        self.metrics['MSE'] = mse
        return mse

    def calculate_rmse(self):
        """Calculate Root Mean Squared Error"""
        rmse = np.sqrt(self.calculate_mse())
        self.metrics['RMSE'] = rmse
        return rmse

    def calculate_ber(self, threshold=1e-6):
        """Calculate Bit Error Rate"""
        differences = np.abs(self.original_vertices - self.stego_vertices)
        binary_orig = (self.original_vertices > threshold).astype(int)
        binary_stego = (self.stego_vertices > threshold).astype(int)

        total_bits = np.prod(binary_orig.shape)
        error_bits = np.sum(binary_orig != binary_stego)

        ber = error_bits / total_bits
        self.metrics['BER'] = ber
        return ber

    def calculate_hausdorff_distance(self):
        """Calculate Hausdorff distance between original and stego models"""
        def directed_hausdorff(source, target):
            tree = KDTree(target)
            distances, _ = tree.query(source)
            return np.max(distances)

        forward = directed_hausdorff(self.original_vertices, self.stego_vertices)
        backward = directed_hausdorff(self.stego_vertices, self.original_vertices)

        hausdorff = max(forward, backward)
        self.metrics['Hausdorff'] = hausdorff
        return hausdorff

    def calculate_histogram_similarity(self, bins=50):
        """Calculate histogram similarity using Wasserstein distance"""
        distances = []

        for i in range(3):
            hist_orig, _ = np.histogram(self.original_vertices[:, i], bins=bins, density=True)
            hist_stego, _ = np.histogram(self.stego_vertices[:, i], bins=bins, density=True)

            distance = wasserstein_distance(hist_orig, hist_stego)
            distances.append(distance)

        hist_similarity = 1 / (1 + np.mean(distances))
        self.metrics['Histogram_Similarity'] = hist_similarity
        return hist_similarity

    def plot_histograms(self, save_path=None):
        """Plot histograms of original and stego models"""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        coords = ['X', 'Y', 'Z']

        for i, (ax, coord) in enumerate(zip(axes, coords)):
            ax.hist(self.original_vertices[:, i], bins=50, alpha=0.5, label='Original', density=True)
            ax.hist(self.stego_vertices[:, i], bins=50, alpha=0.5, label='Stego', density=True)
            ax.set_title(f'{coord} Coordinate Distribution')
            ax.set_xlabel(f'{coord} Value')
            ax.set_ylabel('Density')
            ax.legend()

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def plot_error_distribution(self, save_path=None):
        """Plot the distribution of geometric errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)

        plt.figure(figsize=(10, 6))
        plt.hist(errors, bins=50, density=True)
        plt.title('Geometric Error Distribution')
        plt.xlabel('Error Magnitude')
        plt.ylabel('Density')

        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def evaluate_all(self, plot=True, save_plots=False):
        """Calculate all metrics and optionally generate plots"""
        metrics = {
            'PSNR': self.calculate_psnr(),
            'SSIM': self.calculate_ssim(),
            'MSE': self.calculate_mse(),
            'RMSE': self.calculate_rmse(),
            'BER': self.calculate_ber(),
            'Hausdorff': self.calculate_hausdorff_distance(),
            'Histogram_Similarity': self.calculate_histogram_similarity()
        }

        if plot:
            self.plot_histograms(save_path='/content/drive/MyDrive/OutputPaper2/histograms_RiggedHand_.png' if save_plots else None)
            self.plot_error_distribution(save_path='/content/drive/MyDrive/OutputPaper2/error_distribution_RiggedHand.png' if save_plots else None)
            #self.visualize_error_map(save_path='error_map.png' if save_plots else None)

        return metrics
    def generate_report(self, output_path='/content/drive/MyDrive/OutputPaper2/evaluation_report_RiggedHand.txt'):
        """Generate a detailed evaluation report"""
        with open(output_path, 'w') as f:
            f.write("3D Steganography Evaluation Report\n")
            f.write("=================================\n\n")

            # Model information
            f.write("Model Information:\n")
            f.write(f"Number of vertices: {len(self.original_vertices)}\n")
            f.write(f"Number of faces: {len(self.original_mesh.faces)}\n\n")

            # Metrics
            f.write("Quality Metrics:\n")
            for metric, value in self.metrics.items():
                f.write(f"{metric}: {value:.6f}\n")

            f.write("\nInterpretation:\n")
            f.write("- PSNR > 30 dB typically indicates good quality\n")
            f.write("- SSIM closer to 1 indicates better structural preservation\n")
            f.write("- Lower MSE and RMSE values indicate better similarity\n")
            f.write("- Lower BER indicates better steganographic accuracy\n")
            f.write("- Histogram similarity closer to 1 indicates better statistical imperceptibility\n")

def main():
    # Initialize evaluator
    evaluator = Stego3DEvaluator()

    # Load and align models
    if not evaluator.load_models('/content/drive/MyDrive/OutputPaper2/modified_RiggedHand.obj', '/content/drive/MyDrive/OutputPaper2/RiggedHand.obj', align=True):
        return

    # Perform evaluation
    metrics = evaluator.evaluate_all(plot=True, save_plots=True)

    # Print results
    print("\nEvaluation Results:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.6f}")
    evaluator.generate_report()
    print("\nDetailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report_RiggedHand.txt'")

if __name__ == "__main__":
    main()

Original vertices: 1071
Stego vertices: 1071

Alignment Statistics:
Mean distance: 0.000219
Max distance: 0.001156
Aligned vertices: 1071

Evaluation Results:
PSNR: 77.491462
SSIM: 0.997490
MSE: 0.000000
RMSE: 0.000264
BER: 0.000622
Hausdorff: 0.001156
Histogram_Similarity: 0.702856

Detailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report_RiggedHand.txt'


Embedding for 16834_hand_v1_NEW

In [11]:
#!pip install pycryptodome
#!pip install trimesh
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib
import secrets
import struct
import trimesh
import scipy.spatial as spatial

class SecureFaceSteganography:
    def __init__(self):
        # ... (other code)
        self.LANDMARK_REGIONS = {
            'forehead': [(0.2, 0.8, 0.5), (0.8, 0.8, 0.5)],  # Wider bounding box
            'cheeks': [(0.1, 0.5, 0.5), (0.9, 0.5, 0.5)],  # Wider bounding box
            'jaw': [(0.2, 0.2, 0.5), (0.8, 0.2, 0.5)],  # Wider bounding box
            'nose': [(0.4, 0.6, 0.5), (0.6, 0.6, 0.5)],  # Added nose region
            'chin': [(0.5, 0.1, 0.5)]  #chin region
        }
        # Define SALT_SIZE and BLOCK_SIZE in the __init__ method
        self.SALT_SIZE = 16  # 128 bits for salt
        self.BLOCK_SIZE = 16 # 128 bits for block size
        self.distributed_landmarks = None  # Initialize distributed_landmarks

    def load_face_model(self, file_path):
        """Load 3D face model and prepare it for steganography"""
        try:
            loaded_data = trimesh.load(file_path)
            # Check if loaded_data is a Scene and extract the first mesh if it is
            if isinstance(loaded_data, trimesh.Scene):
                mesh = next(iter(loaded_data.geometry.values())) # Get the first mesh from the scene
            else:
                mesh = loaded_data

            return {
                'vertices': np.array(mesh.vertices),
                'faces': np.array(mesh.faces),
                'normals': np.array(mesh.vertex_normals)
            }
        except Exception as e:
            raise ValueError(f"Error loading face model: {str(e)}")

    def identify_landmark_vertices(self, vertices, landmark_regions=None):
        if landmark_regions is None:
            landmark_regions = self.LANDMARK_REGIONS

        # Use stored landmarks if available, otherwise generate
        if self.distributed_landmarks is None:
            num_landmarks = min(1000, len(vertices))
            self.distributed_landmarks = np.random.choice(len(vertices), num_landmarks, replace=False)

        return sorted(self.distributed_landmarks)  # Return stored landmarks

    def compute_vertex_importance(self, vertices, faces, normals):
        """Compute vertex importance based on geometric features"""
        importance = np.zeros(len(vertices))

        # Calculate curvature (simplified)
        for i, vertex in enumerate(vertices):
            # Find connected vertices
            connected = faces[np.any(faces == i, axis=1)]
            connected = np.unique(connected.flatten())
            connected = connected[connected != i]

            # Calculate local curvature
            if len(connected) > 0:
                connected_vertices = vertices[connected]
                mean_position = np.mean(connected_vertices, axis=0)
                displacement = np.linalg.norm(vertex - mean_position)
                importance[i] = displacement

        return importance

    def generate_passphrase(self, length=32):
        """Generate a cryptographically secure random passphrase"""
        charset = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&*"
        return ''.join(secrets.choice(charset) for _ in range(length))

    def derive_key(self, passphrase, salt):
        """Derive encryption key using SHA-256 with salt"""
        key_material = hashlib.sha256()
        key_material.update(passphrase.encode('utf-8'))
        key_material.update(salt)
        return key_material.digest()

    def encrypt_message(self, message, passphrase):
        """Encrypt a message using AES-128-CBC with secure key derivation"""
        salt = secrets.token_bytes(self.SALT_SIZE)
        key = self.derive_key(passphrase, salt)
        iv = secrets.token_bytes(self.BLOCK_SIZE)

        cipher = AES.new(key, AES.MODE_CBC, iv)
        padded_data = pad(message.encode('utf-8'), self.BLOCK_SIZE)
        ciphertext = cipher.encrypt(padded_data)

        return salt + iv + ciphertext

    def decrypt_message(self, encrypted_data, passphrase):
        """Decrypt a message using AES-128-CBC"""
        salt = encrypted_data[:self.SALT_SIZE]
        iv = encrypted_data[self.SALT_SIZE:self.SALT_SIZE + self.BLOCK_SIZE]
        ciphertext = encrypted_data[self.SALT_SIZE + self.BLOCK_SIZE:]

        key = self.derive_key(passphrase, salt)
        cipher = AES.new(key, AES.MODE_CBC, iv)

        # Attempt to decrypt. Handle potential padding errors
        try:
            padded_message = cipher.decrypt(ciphertext)
            return unpad(padded_message, self.BLOCK_SIZE).decode('utf-8')
        except ValueError:
            print("Warning: Decryption failed. Possible data corruption or incorrect passphrase.")
            return None  # or raise a more specific exception


    def embed_in_face_model(self, face_model, encrypted_data):
        """Embed encrypted data in 3D face model using geometric features"""
        vertices = face_model['vertices'].copy()
        faces = face_model['faces']
        normals = face_model['normals']

        # Get landmarks and importance metrics
        landmark_vertices = self.identify_landmark_vertices(vertices)
        importance = self.compute_vertex_importance(vertices, faces, normals)

        # Convert encrypted data to bit array
        bit_array = ''.join(format(byte, '08b') for byte in encrypted_data)

        # Check if the message is too large
        if len(bit_array) > len(landmark_vertices):
            raise ValueError(f"Message too large for this face model. Message bits: {len(bit_array)}, Available vertices: {len(landmark_vertices)}")

        # Embed data in vertices near landmarks using rounding and a larger scale factor
        for i, bit in enumerate(bit_array):
            vertex_idx = landmark_vertices[i]
            scale_factor = 0.001 * (1 - importance[vertex_idx])  # Significantly increased scale_factor

            # Apply rounding to the modification
            modification = round(float(bit) * scale_factor, 6)  # Round to 6 decimal places

            vertices[vertex_idx] += modification * normals[vertex_idx]

        face_model['vertices'] = vertices
        return face_model

    def extract_from_face_model(self, face_model, original_model, message_length):
        """Extract encrypted data from modified 3D face model"""
        modified_vertices = face_model['vertices']
        original_vertices = original_model['vertices']
        landmark_vertices = self.identify_landmark_vertices(original_vertices)

        bits = []
        for i in range(message_length * 8):  # Iterate over the expected number of bits
            if i >= len(landmark_vertices):
                print(f"Warning: Not enough landmark vertices to extract the full message. Extracted {i} bits, expected {message_length * 8} bits.")
                break  # Stop if we run out of landmark vertices
            vertex_idx = landmark_vertices[i]

            # Compare modified vertex with original
            diff = modified_vertices[vertex_idx] - original_vertices[vertex_idx]
            # Extract bit based on vertex displacement using a tolerance

            extracted_bit = np.dot(diff, original_model['normals'][vertex_idx]) #Using original normals

            #Using tolerance instead of direct comparison with 0
            bit = '1' if extracted_bit > 1e-6 else '0'
            bits.append(bit)

        # Convert bits back to bytes
        extracted_data = bytes(
            int(''.join(bits[i:i + 8]), 2) for i in range(0, len(bits), 8)
        )

        return extracted_data


In [12]:
def main():
    # Initialize steganography system
    stego = SecureFaceSteganography()

    # Load 3D face model (example path)
    face_model = stego.load_face_model('/content/drive/MyDrive/OutputPaper2/16834_hand_v1_NEW.obj')
    original_model = {key: value.copy() for key, value in face_model.items()}

    # Generate secure passphrase
    passphrase = stego.generate_passphrase()
    print(f"Generated passphrase: {passphrase}")

    # Message to hide - Reduced length to fit within the model's capacity
    secret_message = "short"

    # Encrypt the message
    encrypted_data = stego.encrypt_message(secret_message, passphrase)

    # Embed encrypted data in face model
    modified_model = stego.embed_in_face_model(face_model, encrypted_data)

    # Extract and decrypt
    extracted_data = stego.extract_from_face_model(
        modified_model, original_model, len(encrypted_data)
    )
    decrypted_message = stego.decrypt_message(extracted_data, passphrase)

    print(f"Original message: {secret_message}")
    print(f"Decrypted message: {decrypted_message}")

    # Save modified model
    mesh = trimesh.Trimesh(
        vertices=modified_model['vertices'],
        faces=modified_model['faces']
    )
    mesh.export('/content/drive/MyDrive/OutputPaper2/modified_16834_hand_v1_NEW.obj')

if __name__ == "__main__":
    main()

Generated passphrase: 7D^Fm11diBKtbn91h!0u$zRFmbUzv^%&
Original message: short
Decrypted message: short


Evaluation for 16834_hand_v1_NEW

In [13]:
#!pip install pyglet
#!pip install pyglet<2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import trimesh
from scipy.stats import wasserstein_distance
from skimage.metrics import structural_similarity
import math
from scipy.spatial import KDTree
import os

class Stego3DEvaluator:
    def __init__(self):
        """Initialize the 3D steganography evaluator"""
        self.metrics = {}

    def preprocess_models(self, original_mesh, stego_mesh):
        """Preprocess and align models to ensure compatibility"""
        # Center both meshes
        original_mesh.vertices -= original_mesh.vertices.mean(axis=0)
        stego_mesh.vertices -= stego_mesh.vertices.mean(axis=0)

        # Scale to unit cube
        original_scale = np.max(np.abs(original_mesh.vertices))
        stego_scale = np.max(np.abs(stego_mesh.vertices))

        original_mesh.vertices /= original_scale
        stego_mesh.vertices /= stego_scale

        return original_mesh, stego_mesh

    def align_vertices(self, original_vertices, stego_vertices):
        """Align vertices between models using nearest neighbor matching"""
        # Build KD-tree for faster nearest neighbor search
        tree = KDTree(original_vertices)

        # Find nearest neighbors for each stego vertex
        distances, indices = tree.query(stego_vertices)

        # Create aligned vertex arrays
        aligned_original = original_vertices[indices]
        aligned_stego = stego_vertices

        return aligned_original, aligned_stego, distances

    def load_models(self, original_path, stego_path, align=True):
        """Load and preprocess original and stego 3D models"""
        try:
            # Load meshes
            original_mesh = trimesh.load(original_path)
            stego_mesh = trimesh.load(stego_path)

            # Check if loaded data is a Scene and extract the first mesh if it is
            if isinstance(original_mesh, trimesh.Scene):
                original_mesh = next(iter(original_mesh.geometry.values())) # Get the first mesh from the scene
            if isinstance(stego_mesh, trimesh.Scene):
                stego_mesh = next(iter(stego_mesh.geometry.values())) # Get the first mesh from the scene

            print(f"Original vertices: {len(original_mesh.vertices)}")
            print(f"Stego vertices: {len(stego_mesh.vertices)}")

            # Preprocess meshes
            original_mesh, stego_mesh = self.preprocess_models(original_mesh, stego_mesh)

            if align:
                # Align vertices
                self.original_vertices, self.stego_vertices, distances = self.align_vertices(
                    original_mesh.vertices,
                    stego_mesh.vertices
                )

                # Print alignment statistics
                print(f"\nAlignment Statistics:")
                print(f"Mean distance: {np.mean(distances):.6f}")
                print(f"Max distance: {np.max(distances):.6f}")
                print(f"Aligned vertices: {len(self.original_vertices)}")
            else:
                if len(original_mesh.vertices) != len(stego_mesh.vertices):
                    raise ValueError("Models have different number of vertices and alignment is disabled")
                self.original_vertices = original_mesh.vertices
                self.stego_vertices = stego_mesh.vertices

            self.original_mesh = original_mesh
            self.stego_mesh = stego_mesh

            return True

        except Exception as e:
            print(f"Error loading models: {str(e)}")
            return False

    def calculate_vertex_error_map(self):
        """Calculate and visualize vertex-wise errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)
        return errors



    def calculate_psnr(self):
        """Calculate Peak Signal-to-Noise Ratio"""
        mse = np.mean((self.original_vertices - self.stego_vertices) ** 2)
        if mse == 0:
            return float('inf')

        max_val = np.max(self.original_vertices) - np.min(self.original_vertices)
        psnr = 20 * math.log10(max_val / math.sqrt(mse))
        self.metrics['PSNR'] = psnr
        return psnr

    def calculate_ssim(self):
        """Calculate Structural Similarity Index"""
        orig_reshaped = self.original_vertices.reshape(-1, 3)
        stego_reshaped = self.stego_vertices.reshape(-1, 3)

        ssim_scores = []
        for i in range(3):
            score = structural_similarity(
                orig_reshaped[:, i],
                stego_reshaped[:, i],
                data_range=np.max(orig_reshaped[:, i]) - np.min(orig_reshaped[:, i])
            )
            ssim_scores.append(score)

        ssim = np.mean(ssim_scores)
        self.metrics['SSIM'] = ssim
        return ssim

    def calculate_mse(self):
        """Calculate Mean Squared Error"""
        mse = mean_squared_error(self.original_vertices, self.stego_vertices)
        self.metrics['MSE'] = mse
        return mse

    def calculate_rmse(self):
        """Calculate Root Mean Squared Error"""
        rmse = np.sqrt(self.calculate_mse())
        self.metrics['RMSE'] = rmse
        return rmse

    def calculate_ber(self, threshold=1e-6):
        """Calculate Bit Error Rate"""
        differences = np.abs(self.original_vertices - self.stego_vertices)
        binary_orig = (self.original_vertices > threshold).astype(int)
        binary_stego = (self.stego_vertices > threshold).astype(int)

        total_bits = np.prod(binary_orig.shape)
        error_bits = np.sum(binary_orig != binary_stego)

        ber = error_bits / total_bits
        self.metrics['BER'] = ber
        return ber

    def calculate_hausdorff_distance(self):
        """Calculate Hausdorff distance between original and stego models"""
        def directed_hausdorff(source, target):
            tree = KDTree(target)
            distances, _ = tree.query(source)
            return np.max(distances)

        forward = directed_hausdorff(self.original_vertices, self.stego_vertices)
        backward = directed_hausdorff(self.stego_vertices, self.original_vertices)

        hausdorff = max(forward, backward)
        self.metrics['Hausdorff'] = hausdorff
        return hausdorff

    def calculate_histogram_similarity(self, bins=50):
        """Calculate histogram similarity using Wasserstein distance"""
        distances = []

        for i in range(3):
            hist_orig, _ = np.histogram(self.original_vertices[:, i], bins=bins, density=True)
            hist_stego, _ = np.histogram(self.stego_vertices[:, i], bins=bins, density=True)

            distance = wasserstein_distance(hist_orig, hist_stego)
            distances.append(distance)

        hist_similarity = 1 / (1 + np.mean(distances))
        self.metrics['Histogram_Similarity'] = hist_similarity
        return hist_similarity

    def plot_histograms(self, save_path=None):
        """Plot histograms of original and stego models"""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        coords = ['X', 'Y', 'Z']

        for i, (ax, coord) in enumerate(zip(axes, coords)):
            ax.hist(self.original_vertices[:, i], bins=50, alpha=0.5, label='Original', density=True)
            ax.hist(self.stego_vertices[:, i], bins=50, alpha=0.5, label='Stego', density=True)
            ax.set_title(f'{coord} Coordinate Distribution')
            ax.set_xlabel(f'{coord} Value')
            ax.set_ylabel('Density')
            ax.legend()

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def plot_error_distribution(self, save_path=None):
        """Plot the distribution of geometric errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)

        plt.figure(figsize=(10, 6))
        plt.hist(errors, bins=50, density=True)
        plt.title('Geometric Error Distribution')
        plt.xlabel('Error Magnitude')
        plt.ylabel('Density')

        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def evaluate_all(self, plot=True, save_plots=False):
        """Calculate all metrics and optionally generate plots"""
        metrics = {
            'PSNR': self.calculate_psnr(),
            'SSIM': self.calculate_ssim(),
            'MSE': self.calculate_mse(),
            'RMSE': self.calculate_rmse(),
            'BER': self.calculate_ber(),
            'Hausdorff': self.calculate_hausdorff_distance(),
            'Histogram_Similarity': self.calculate_histogram_similarity()
        }

        if plot:
            self.plot_histograms(save_path='/content/drive/MyDrive/OutputPaper2/histograms_16834_hand_v1_NEW_.png' if save_plots else None)
            self.plot_error_distribution(save_path='/content/drive/MyDrive/OutputPaper2/error_distribution_16834_hand_v1_NEW.png' if save_plots else None)
            #self.visualize_error_map(save_path='error_map.png' if save_plots else None)

        return metrics
    def generate_report(self, output_path='/content/drive/MyDrive/OutputPaper2/evaluation_report_16834_hand_v1_NEW.txt'):
        """Generate a detailed evaluation report"""
        with open(output_path, 'w') as f:
            f.write("3D Steganography Evaluation Report\n")
            f.write("=================================\n\n")

            # Model information
            f.write("Model Information:\n")
            f.write(f"Number of vertices: {len(self.original_vertices)}\n")
            f.write(f"Number of faces: {len(self.original_mesh.faces)}\n\n")

            # Metrics
            f.write("Quality Metrics:\n")
            for metric, value in self.metrics.items():
                f.write(f"{metric}: {value:.6f}\n")

            f.write("\nInterpretation:\n")
            f.write("- PSNR > 30 dB typically indicates good quality\n")
            f.write("- SSIM closer to 1 indicates better structural preservation\n")
            f.write("- Lower MSE and RMSE values indicate better similarity\n")
            f.write("- Lower BER indicates better steganographic accuracy\n")
            f.write("- Histogram similarity closer to 1 indicates better statistical imperceptibility\n")

def main():
    # Initialize evaluator
    evaluator = Stego3DEvaluator()

    # Load and align models
    if not evaluator.load_models('/content/drive/MyDrive/OutputPaper2/modified_16834_hand_v1_NEW.obj', '/content/drive/MyDrive/OutputPaper2/16834_hand_v1_NEW.obj', align=True):
        return

    # Perform evaluation
    metrics = evaluator.evaluate_all(plot=True, save_plots=True)

    # Print results
    print("\nEvaluation Results:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.6f}")
    evaluator.generate_report()
    print("\nDetailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report_16834_hand_v1_NEW.txt'")

if __name__ == "__main__":
    main()

Original vertices: 66848
Stego vertices: 66848

Alignment Statistics:
Mean distance: 0.000000
Max distance: 0.000081
Aligned vertices: 66848

Evaluation Results:
PSNR: 115.993959
SSIM: 1.000000
MSE: 0.000000
RMSE: 0.000003
BER: 0.000000
Hausdorff: 0.000081
Histogram_Similarity: 0.999960

Detailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report_16834_hand_v1_NEW.txt'


Embedding for 15807_Zombie_Hand_v1_NEW

In [15]:
#!pip install pycryptodome
#!pip install trimesh
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib
import secrets
import struct
import trimesh
import scipy.spatial as spatial

class SecureFaceSteganography:
    def __init__(self):
        # ... (other code)
        self.LANDMARK_REGIONS = {
            'forehead': [(0.2, 0.8, 0.5), (0.8, 0.8, 0.5)],  # Wider bounding box
            'cheeks': [(0.1, 0.5, 0.5), (0.9, 0.5, 0.5)],  # Wider bounding box
            'jaw': [(0.2, 0.2, 0.5), (0.8, 0.2, 0.5)],  # Wider bounding box
            'nose': [(0.4, 0.6, 0.5), (0.6, 0.6, 0.5)],  # Added nose region
            'chin': [(0.5, 0.1, 0.5)]  #chin region
        }
        # Define SALT_SIZE and BLOCK_SIZE in the __init__ method
        self.SALT_SIZE = 16  # 128 bits for salt
        self.BLOCK_SIZE = 16 # 128 bits for block size
        self.distributed_landmarks = None  # Initialize distributed_landmarks

    def load_face_model(self, file_path):
        """Load 3D face model and prepare it for steganography"""
        try:
            loaded_data = trimesh.load(file_path)
            # Check if loaded_data is a Scene and extract the first mesh if it is
            if isinstance(loaded_data, trimesh.Scene):
                mesh = next(iter(loaded_data.geometry.values())) # Get the first mesh from the scene
            else:
                mesh = loaded_data

            return {
                'vertices': np.array(mesh.vertices),
                'faces': np.array(mesh.faces),
                'normals': np.array(mesh.vertex_normals)
            }
        except Exception as e:
            raise ValueError(f"Error loading face model: {str(e)}")

    def identify_landmark_vertices(self, vertices, landmark_regions=None):
        if landmark_regions is None:
            landmark_regions = self.LANDMARK_REGIONS

        # Use stored landmarks if available, otherwise generate
        if self.distributed_landmarks is None:
            num_landmarks = min(1000, len(vertices))
            self.distributed_landmarks = np.random.choice(len(vertices), num_landmarks, replace=False)

        return sorted(self.distributed_landmarks)  # Return stored landmarks

    def compute_vertex_importance(self, vertices, faces, normals):
        """Compute vertex importance based on geometric features"""
        importance = np.zeros(len(vertices))

        # Calculate curvature (simplified)
        for i, vertex in enumerate(vertices):
            # Find connected vertices
            connected = faces[np.any(faces == i, axis=1)]
            connected = np.unique(connected.flatten())
            connected = connected[connected != i]

            # Calculate local curvature
            if len(connected) > 0:
                connected_vertices = vertices[connected]
                mean_position = np.mean(connected_vertices, axis=0)
                displacement = np.linalg.norm(vertex - mean_position)
                importance[i] = displacement

        return importance

    def generate_passphrase(self, length=32):
        """Generate a cryptographically secure random passphrase"""
        charset = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&*"
        return ''.join(secrets.choice(charset) for _ in range(length))

    def derive_key(self, passphrase, salt):
        """Derive encryption key using SHA-256 with salt"""
        key_material = hashlib.sha256()
        key_material.update(passphrase.encode('utf-8'))
        key_material.update(salt)
        return key_material.digest()

    def encrypt_message(self, message, passphrase):
        """Encrypt a message using AES-128-CBC with secure key derivation"""
        salt = secrets.token_bytes(self.SALT_SIZE)
        key = self.derive_key(passphrase, salt)
        iv = secrets.token_bytes(self.BLOCK_SIZE)

        cipher = AES.new(key, AES.MODE_CBC, iv)
        padded_data = pad(message.encode('utf-8'), self.BLOCK_SIZE)
        ciphertext = cipher.encrypt(padded_data)

        return salt + iv + ciphertext

    def decrypt_message(self, encrypted_data, passphrase):
        """Decrypt a message using AES-128-CBC"""
        salt = encrypted_data[:self.SALT_SIZE]
        iv = encrypted_data[self.SALT_SIZE:self.SALT_SIZE + self.BLOCK_SIZE]
        ciphertext = encrypted_data[self.SALT_SIZE + self.BLOCK_SIZE:]

        key = self.derive_key(passphrase, salt)
        cipher = AES.new(key, AES.MODE_CBC, iv)

        # Attempt to decrypt. Handle potential padding errors
        try:
            padded_message = cipher.decrypt(ciphertext)
            return unpad(padded_message, self.BLOCK_SIZE).decode('utf-8')
        except ValueError:
            print("Warning: Decryption failed. Possible data corruption or incorrect passphrase.")
            return None  # or raise a more specific exception


    def embed_in_face_model(self, face_model, encrypted_data):
        """Embed encrypted data in 3D face model using geometric features"""
        vertices = face_model['vertices'].copy()
        faces = face_model['faces']
        normals = face_model['normals']

        # Get landmarks and importance metrics
        landmark_vertices = self.identify_landmark_vertices(vertices)
        importance = self.compute_vertex_importance(vertices, faces, normals)

        # Convert encrypted data to bit array
        bit_array = ''.join(format(byte, '08b') for byte in encrypted_data)

        # Check if the message is too large
        if len(bit_array) > len(landmark_vertices):
            raise ValueError(f"Message too large for this face model. Message bits: {len(bit_array)}, Available vertices: {len(landmark_vertices)}")

        # Embed data in vertices near landmarks using rounding and a larger scale factor
        for i, bit in enumerate(bit_array):
            vertex_idx = landmark_vertices[i]
            scale_factor = 0.001 * (1 - importance[vertex_idx])  # Significantly increased scale_factor

            # Apply rounding to the modification
            modification = round(float(bit) * scale_factor, 6)  # Round to 6 decimal places

            vertices[vertex_idx] += modification * normals[vertex_idx]

        face_model['vertices'] = vertices
        return face_model

    def extract_from_face_model(self, face_model, original_model, message_length):
        """Extract encrypted data from modified 3D face model"""
        modified_vertices = face_model['vertices']
        original_vertices = original_model['vertices']
        landmark_vertices = self.identify_landmark_vertices(original_vertices)

        bits = []
        for i in range(message_length * 8):  # Iterate over the expected number of bits
            if i >= len(landmark_vertices):
                print(f"Warning: Not enough landmark vertices to extract the full message. Extracted {i} bits, expected {message_length * 8} bits.")
                break  # Stop if we run out of landmark vertices
            vertex_idx = landmark_vertices[i]

            # Compare modified vertex with original
            diff = modified_vertices[vertex_idx] - original_vertices[vertex_idx]
            # Extract bit based on vertex displacement using a tolerance

            extracted_bit = np.dot(diff, original_model['normals'][vertex_idx]) #Using original normals

            #Using tolerance instead of direct comparison with 0
            bit = '1' if extracted_bit > 1e-6 else '0'
            bits.append(bit)

        # Convert bits back to bytes
        extracted_data = bytes(
            int(''.join(bits[i:i + 8]), 2) for i in range(0, len(bits), 8)
        )

        return extracted_data


In [16]:
def main():
    # Initialize steganography system
    stego = SecureFaceSteganography()

    # Load 3D face model (example path)
    face_model = stego.load_face_model('/content/drive/MyDrive/OutputPaper2/15807_Zombie_Hand_v1_NEW.obj')
    original_model = {key: value.copy() for key, value in face_model.items()}

    # Generate secure passphrase
    passphrase = stego.generate_passphrase()
    print(f"Generated passphrase: {passphrase}")

    # Message to hide - Reduced length to fit within the model's capacity
    secret_message = "short"

    # Encrypt the message
    encrypted_data = stego.encrypt_message(secret_message, passphrase)

    # Embed encrypted data in face model
    modified_model = stego.embed_in_face_model(face_model, encrypted_data)

    # Extract and decrypt
    extracted_data = stego.extract_from_face_model(
        modified_model, original_model, len(encrypted_data)
    )
    decrypted_message = stego.decrypt_message(extracted_data, passphrase)

    print(f"Original message: {secret_message}")
    print(f"Decrypted message: {decrypted_message}")

    # Save modified model
    mesh = trimesh.Trimesh(
        vertices=modified_model['vertices'],
        faces=modified_model['faces']
    )
    mesh.export('/content/drive/MyDrive/OutputPaper2/modified_15807_Zombie_Hand_v1_NEW.obj')

if __name__ == "__main__":
    main()

Generated passphrase: IRyZX3vV9vg1M%VSkVBiHDyPhL#euGm@
Original message: short
Decrypted message: short


Evaluation for 15807_Zombie_Hand_v1_NEW

In [17]:
#!pip install pyglet
#!pip install pyglet<2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import trimesh
from scipy.stats import wasserstein_distance
from skimage.metrics import structural_similarity
import math
from scipy.spatial import KDTree
import os

class Stego3DEvaluator:
    def __init__(self):
        """Initialize the 3D steganography evaluator"""
        self.metrics = {}

    def preprocess_models(self, original_mesh, stego_mesh):
        """Preprocess and align models to ensure compatibility"""
        # Center both meshes
        original_mesh.vertices -= original_mesh.vertices.mean(axis=0)
        stego_mesh.vertices -= stego_mesh.vertices.mean(axis=0)

        # Scale to unit cube
        original_scale = np.max(np.abs(original_mesh.vertices))
        stego_scale = np.max(np.abs(stego_mesh.vertices))

        original_mesh.vertices /= original_scale
        stego_mesh.vertices /= stego_scale

        return original_mesh, stego_mesh

    def align_vertices(self, original_vertices, stego_vertices):
        """Align vertices between models using nearest neighbor matching"""
        # Build KD-tree for faster nearest neighbor search
        tree = KDTree(original_vertices)

        # Find nearest neighbors for each stego vertex
        distances, indices = tree.query(stego_vertices)

        # Create aligned vertex arrays
        aligned_original = original_vertices[indices]
        aligned_stego = stego_vertices

        return aligned_original, aligned_stego, distances

    def load_models(self, original_path, stego_path, align=True):
        """Load and preprocess original and stego 3D models"""
        try:
            # Load meshes
            original_mesh = trimesh.load(original_path)
            stego_mesh = trimesh.load(stego_path)

            # Check if loaded data is a Scene and extract the first mesh if it is
            if isinstance(original_mesh, trimesh.Scene):
                original_mesh = next(iter(original_mesh.geometry.values())) # Get the first mesh from the scene
            if isinstance(stego_mesh, trimesh.Scene):
                stego_mesh = next(iter(stego_mesh.geometry.values())) # Get the first mesh from the scene

            print(f"Original vertices: {len(original_mesh.vertices)}")
            print(f"Stego vertices: {len(stego_mesh.vertices)}")

            # Preprocess meshes
            original_mesh, stego_mesh = self.preprocess_models(original_mesh, stego_mesh)

            if align:
                # Align vertices
                self.original_vertices, self.stego_vertices, distances = self.align_vertices(
                    original_mesh.vertices,
                    stego_mesh.vertices
                )

                # Print alignment statistics
                print(f"\nAlignment Statistics:")
                print(f"Mean distance: {np.mean(distances):.6f}")
                print(f"Max distance: {np.max(distances):.6f}")
                print(f"Aligned vertices: {len(self.original_vertices)}")
            else:
                if len(original_mesh.vertices) != len(stego_mesh.vertices):
                    raise ValueError("Models have different number of vertices and alignment is disabled")
                self.original_vertices = original_mesh.vertices
                self.stego_vertices = stego_mesh.vertices

            self.original_mesh = original_mesh
            self.stego_mesh = stego_mesh

            return True

        except Exception as e:
            print(f"Error loading models: {str(e)}")
            return False

    def calculate_vertex_error_map(self):
        """Calculate and visualize vertex-wise errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)
        return errors



    def calculate_psnr(self):
        """Calculate Peak Signal-to-Noise Ratio"""
        mse = np.mean((self.original_vertices - self.stego_vertices) ** 2)
        if mse == 0:
            return float('inf')

        max_val = np.max(self.original_vertices) - np.min(self.original_vertices)
        psnr = 20 * math.log10(max_val / math.sqrt(mse))
        self.metrics['PSNR'] = psnr
        return psnr

    def calculate_ssim(self):
        """Calculate Structural Similarity Index"""
        orig_reshaped = self.original_vertices.reshape(-1, 3)
        stego_reshaped = self.stego_vertices.reshape(-1, 3)

        ssim_scores = []
        for i in range(3):
            score = structural_similarity(
                orig_reshaped[:, i],
                stego_reshaped[:, i],
                data_range=np.max(orig_reshaped[:, i]) - np.min(orig_reshaped[:, i])
            )
            ssim_scores.append(score)

        ssim = np.mean(ssim_scores)
        self.metrics['SSIM'] = ssim
        return ssim

    def calculate_mse(self):
        """Calculate Mean Squared Error"""
        mse = mean_squared_error(self.original_vertices, self.stego_vertices)
        self.metrics['MSE'] = mse
        return mse

    def calculate_rmse(self):
        """Calculate Root Mean Squared Error"""
        rmse = np.sqrt(self.calculate_mse())
        self.metrics['RMSE'] = rmse
        return rmse

    def calculate_ber(self, threshold=1e-6):
        """Calculate Bit Error Rate"""
        differences = np.abs(self.original_vertices - self.stego_vertices)
        binary_orig = (self.original_vertices > threshold).astype(int)
        binary_stego = (self.stego_vertices > threshold).astype(int)

        total_bits = np.prod(binary_orig.shape)
        error_bits = np.sum(binary_orig != binary_stego)

        ber = error_bits / total_bits
        self.metrics['BER'] = ber
        return ber

    def calculate_hausdorff_distance(self):
        """Calculate Hausdorff distance between original and stego models"""
        def directed_hausdorff(source, target):
            tree = KDTree(target)
            distances, _ = tree.query(source)
            return np.max(distances)

        forward = directed_hausdorff(self.original_vertices, self.stego_vertices)
        backward = directed_hausdorff(self.stego_vertices, self.original_vertices)

        hausdorff = max(forward, backward)
        self.metrics['Hausdorff'] = hausdorff
        return hausdorff

    def calculate_histogram_similarity(self, bins=50):
        """Calculate histogram similarity using Wasserstein distance"""
        distances = []

        for i in range(3):
            hist_orig, _ = np.histogram(self.original_vertices[:, i], bins=bins, density=True)
            hist_stego, _ = np.histogram(self.stego_vertices[:, i], bins=bins, density=True)

            distance = wasserstein_distance(hist_orig, hist_stego)
            distances.append(distance)

        hist_similarity = 1 / (1 + np.mean(distances))
        self.metrics['Histogram_Similarity'] = hist_similarity
        return hist_similarity

    def plot_histograms(self, save_path=None):
        """Plot histograms of original and stego models"""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        coords = ['X', 'Y', 'Z']

        for i, (ax, coord) in enumerate(zip(axes, coords)):
            ax.hist(self.original_vertices[:, i], bins=50, alpha=0.5, label='Original', density=True)
            ax.hist(self.stego_vertices[:, i], bins=50, alpha=0.5, label='Stego', density=True)
            ax.set_title(f'{coord} Coordinate Distribution')
            ax.set_xlabel(f'{coord} Value')
            ax.set_ylabel('Density')
            ax.legend()

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def plot_error_distribution(self, save_path=None):
        """Plot the distribution of geometric errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)

        plt.figure(figsize=(10, 6))
        plt.hist(errors, bins=50, density=True)
        plt.title('Geometric Error Distribution')
        plt.xlabel('Error Magnitude')
        plt.ylabel('Density')

        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def evaluate_all(self, plot=True, save_plots=False):
        """Calculate all metrics and optionally generate plots"""
        metrics = {
            'PSNR': self.calculate_psnr(),
            'SSIM': self.calculate_ssim(),
            'MSE': self.calculate_mse(),
            'RMSE': self.calculate_rmse(),
            'BER': self.calculate_ber(),
            'Hausdorff': self.calculate_hausdorff_distance(),
            'Histogram_Similarity': self.calculate_histogram_similarity()
        }

        if plot:
            self.plot_histograms(save_path='/content/drive/MyDrive/OutputPaper2/histograms_15807_Zombie_Hand_v1_NEW_.png' if save_plots else None)
            self.plot_error_distribution(save_path='/content/drive/MyDrive/OutputPaper2/error_distribution_15807_Zombie_Hand_v1_NEW.png' if save_plots else None)
            #self.visualize_error_map(save_path='error_map.png' if save_plots else None)

        return metrics
    def generate_report(self, output_path='/content/drive/MyDrive/OutputPaper2/evaluation_report_15807_Zombie_Hand_v1_NEW.txt'):
        """Generate a detailed evaluation report"""
        with open(output_path, 'w') as f:
            f.write("3D Steganography Evaluation Report\n")
            f.write("=================================\n\n")

            # Model information
            f.write("Model Information:\n")
            f.write(f"Number of vertices: {len(self.original_vertices)}\n")
            f.write(f"Number of faces: {len(self.original_mesh.faces)}\n\n")

            # Metrics
            f.write("Quality Metrics:\n")
            for metric, value in self.metrics.items():
                f.write(f"{metric}: {value:.6f}\n")

            f.write("\nInterpretation:\n")
            f.write("- PSNR > 30 dB typically indicates good quality\n")
            f.write("- SSIM closer to 1 indicates better structural preservation\n")
            f.write("- Lower MSE and RMSE values indicate better similarity\n")
            f.write("- Lower BER indicates better steganographic accuracy\n")
            f.write("- Histogram similarity closer to 1 indicates better statistical imperceptibility\n")

def main():
    # Initialize evaluator
    evaluator = Stego3DEvaluator()

    # Load and align models
    if not evaluator.load_models('/content/drive/MyDrive/OutputPaper2/modified_15807_Zombie_Hand_v1_NEW.obj', '/content/drive/MyDrive/OutputPaper2/15807_Zombie_Hand_v1_NEW.obj', align=True):
        return

    # Perform evaluation
    metrics = evaluator.evaluate_all(plot=True, save_plots=True)

    # Print results
    print("\nEvaluation Results:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.6f}")
    evaluator.generate_report()
    print("\nDetailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report_15807_Zombie_Hand_v1_NEW.txt'")

if __name__ == "__main__":
    main()

Original vertices: 17888
Stego vertices: 17961

Alignment Statistics:
Mean distance: 0.002351
Max distance: 0.003575
Aligned vertices: 17961

Evaluation Results:
PSNR: 59.764732
SSIM: 0.998935
MSE: 0.000002
RMSE: 0.001430
BER: 0.001466
Hausdorff: 0.003575
Histogram_Similarity: 0.984463

Detailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report_15807_Zombie_Hand_v1_NEW.txt'


Embedding for 15800_Mummy_Hand_v1_NEW

In [18]:
#!pip install pycryptodome
#!pip install trimesh
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib
import secrets
import struct
import trimesh
import scipy.spatial as spatial

class SecureFaceSteganography:
    def __init__(self):
        # ... (other code)
        self.LANDMARK_REGIONS = {
            'forehead': [(0.2, 0.8, 0.5), (0.8, 0.8, 0.5)],  # Wider bounding box
            'cheeks': [(0.1, 0.5, 0.5), (0.9, 0.5, 0.5)],  # Wider bounding box
            'jaw': [(0.2, 0.2, 0.5), (0.8, 0.2, 0.5)],  # Wider bounding box
            'nose': [(0.4, 0.6, 0.5), (0.6, 0.6, 0.5)],  # Added nose region
            'chin': [(0.5, 0.1, 0.5)]  #chin region
        }
        # Define SALT_SIZE and BLOCK_SIZE in the __init__ method
        self.SALT_SIZE = 16  # 128 bits for salt
        self.BLOCK_SIZE = 16 # 128 bits for block size
        self.distributed_landmarks = None  # Initialize distributed_landmarks

    def load_face_model(self, file_path):
        """Load 3D face model and prepare it for steganography"""
        try:
            loaded_data = trimesh.load(file_path)
            # Check if loaded_data is a Scene and extract the first mesh if it is
            if isinstance(loaded_data, trimesh.Scene):
                mesh = next(iter(loaded_data.geometry.values())) # Get the first mesh from the scene
            else:
                mesh = loaded_data

            return {
                'vertices': np.array(mesh.vertices),
                'faces': np.array(mesh.faces),
                'normals': np.array(mesh.vertex_normals)
            }
        except Exception as e:
            raise ValueError(f"Error loading face model: {str(e)}")

    def identify_landmark_vertices(self, vertices, landmark_regions=None):
        if landmark_regions is None:
            landmark_regions = self.LANDMARK_REGIONS

        # Use stored landmarks if available, otherwise generate
        if self.distributed_landmarks is None:
            num_landmarks = min(1000, len(vertices))
            self.distributed_landmarks = np.random.choice(len(vertices), num_landmarks, replace=False)

        return sorted(self.distributed_landmarks)  # Return stored landmarks

    def compute_vertex_importance(self, vertices, faces, normals):
        """Compute vertex importance based on geometric features"""
        importance = np.zeros(len(vertices))

        # Calculate curvature (simplified)
        for i, vertex in enumerate(vertices):
            # Find connected vertices
            connected = faces[np.any(faces == i, axis=1)]
            connected = np.unique(connected.flatten())
            connected = connected[connected != i]

            # Calculate local curvature
            if len(connected) > 0:
                connected_vertices = vertices[connected]
                mean_position = np.mean(connected_vertices, axis=0)
                displacement = np.linalg.norm(vertex - mean_position)
                importance[i] = displacement

        return importance

    def generate_passphrase(self, length=32):
        """Generate a cryptographically secure random passphrase"""
        charset = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&*"
        return ''.join(secrets.choice(charset) for _ in range(length))

    def derive_key(self, passphrase, salt):
        """Derive encryption key using SHA-256 with salt"""
        key_material = hashlib.sha256()
        key_material.update(passphrase.encode('utf-8'))
        key_material.update(salt)
        return key_material.digest()

    def encrypt_message(self, message, passphrase):
        """Encrypt a message using AES-128-CBC with secure key derivation"""
        salt = secrets.token_bytes(self.SALT_SIZE)
        key = self.derive_key(passphrase, salt)
        iv = secrets.token_bytes(self.BLOCK_SIZE)

        cipher = AES.new(key, AES.MODE_CBC, iv)
        padded_data = pad(message.encode('utf-8'), self.BLOCK_SIZE)
        ciphertext = cipher.encrypt(padded_data)

        return salt + iv + ciphertext

    def decrypt_message(self, encrypted_data, passphrase):
        """Decrypt a message using AES-128-CBC"""
        salt = encrypted_data[:self.SALT_SIZE]
        iv = encrypted_data[self.SALT_SIZE:self.SALT_SIZE + self.BLOCK_SIZE]
        ciphertext = encrypted_data[self.SALT_SIZE + self.BLOCK_SIZE:]

        key = self.derive_key(passphrase, salt)
        cipher = AES.new(key, AES.MODE_CBC, iv)

        # Attempt to decrypt. Handle potential padding errors
        try:
            padded_message = cipher.decrypt(ciphertext)
            return unpad(padded_message, self.BLOCK_SIZE).decode('utf-8')
        except ValueError:
            print("Warning: Decryption failed. Possible data corruption or incorrect passphrase.")
            return None  # or raise a more specific exception


    def embed_in_face_model(self, face_model, encrypted_data):
        """Embed encrypted data in 3D face model using geometric features"""
        vertices = face_model['vertices'].copy()
        faces = face_model['faces']
        normals = face_model['normals']

        # Get landmarks and importance metrics
        landmark_vertices = self.identify_landmark_vertices(vertices)
        importance = self.compute_vertex_importance(vertices, faces, normals)

        # Convert encrypted data to bit array
        bit_array = ''.join(format(byte, '08b') for byte in encrypted_data)

        # Check if the message is too large
        if len(bit_array) > len(landmark_vertices):
            raise ValueError(f"Message too large for this face model. Message bits: {len(bit_array)}, Available vertices: {len(landmark_vertices)}")

        # Embed data in vertices near landmarks using rounding and a larger scale factor
        for i, bit in enumerate(bit_array):
            vertex_idx = landmark_vertices[i]
            scale_factor = 0.001 * (1 - importance[vertex_idx])  # Significantly increased scale_factor

            # Apply rounding to the modification
            modification = round(float(bit) * scale_factor, 6)  # Round to 6 decimal places

            vertices[vertex_idx] += modification * normals[vertex_idx]

        face_model['vertices'] = vertices
        return face_model

    def extract_from_face_model(self, face_model, original_model, message_length):
        """Extract encrypted data from modified 3D face model"""
        modified_vertices = face_model['vertices']
        original_vertices = original_model['vertices']
        landmark_vertices = self.identify_landmark_vertices(original_vertices)

        bits = []
        for i in range(message_length * 8):  # Iterate over the expected number of bits
            if i >= len(landmark_vertices):
                print(f"Warning: Not enough landmark vertices to extract the full message. Extracted {i} bits, expected {message_length * 8} bits.")
                break  # Stop if we run out of landmark vertices
            vertex_idx = landmark_vertices[i]

            # Compare modified vertex with original
            diff = modified_vertices[vertex_idx] - original_vertices[vertex_idx]
            # Extract bit based on vertex displacement using a tolerance

            extracted_bit = np.dot(diff, original_model['normals'][vertex_idx]) #Using original normals

            #Using tolerance instead of direct comparison with 0
            bit = '1' if extracted_bit > 1e-6 else '0'
            bits.append(bit)

        # Convert bits back to bytes
        extracted_data = bytes(
            int(''.join(bits[i:i + 8]), 2) for i in range(0, len(bits), 8)
        )

        return extracted_data


In [19]:
def main():
    # Initialize steganography system
    stego = SecureFaceSteganography()

    # Load 3D face model (example path)
    face_model = stego.load_face_model('/content/drive/MyDrive/OutputPaper2/15800_Mummy_Hand_v1_NEW.obj')
    original_model = {key: value.copy() for key, value in face_model.items()}

    # Generate secure passphrase
    passphrase = stego.generate_passphrase()
    print(f"Generated passphrase: {passphrase}")

    # Message to hide - Reduced length to fit within the model's capacity
    secret_message = "short"

    # Encrypt the message
    encrypted_data = stego.encrypt_message(secret_message, passphrase)

    # Embed encrypted data in face model
    modified_model = stego.embed_in_face_model(face_model, encrypted_data)

    # Extract and decrypt
    extracted_data = stego.extract_from_face_model(
        modified_model, original_model, len(encrypted_data)
    )
    decrypted_message = stego.decrypt_message(extracted_data, passphrase)

    print(f"Original message: {secret_message}")
    print(f"Decrypted message: {decrypted_message}")

    # Save modified model
    mesh = trimesh.Trimesh(
        vertices=modified_model['vertices'],
        faces=modified_model['faces']
    )
    mesh.export('/content/drive/MyDrive/OutputPaper2/modified_15800_Mummy_Hand_v1_NEW.obj')

if __name__ == "__main__":
    main()

Generated passphrase: a2LK6QRjf*dF06fZFt3iEC@r0ncxvxxd
Original message: short
Decrypted message: short


Embedding for 15742_Zombie_Arm_v1

In [21]:
def main():
    # Initialize steganography system
    stego = SecureFaceSteganography()

    # Load 3D face model (example path)
    face_model = stego.load_face_model('/content/drive/MyDrive/OutputPaper2/15742_Zombie_Arm_v1.obj')
    original_model = {key: value.copy() for key, value in face_model.items()}

    # Generate secure passphrase
    passphrase = stego.generate_passphrase()
    print(f"Generated passphrase: {passphrase}")

    # Message to hide - Reduced length to fit within the model's capacity
    secret_message = "short"

    # Encrypt the message
    encrypted_data = stego.encrypt_message(secret_message, passphrase)

    # Embed encrypted data in face model
    modified_model = stego.embed_in_face_model(face_model, encrypted_data)

    # Extract and decrypt
    extracted_data = stego.extract_from_face_model(
        modified_model, original_model, len(encrypted_data)
    )
    decrypted_message = stego.decrypt_message(extracted_data, passphrase)

    print(f"Original message: {secret_message}")
    print(f"Decrypted message: {decrypted_message}")

    # Save modified model
    mesh = trimesh.Trimesh(
        vertices=modified_model['vertices'],
        faces=modified_model['faces']
    )
    mesh.export('/content/drive/MyDrive/OutputPaper2/modified_15742_Zombie_Arm_v1.obj')

if __name__ == "__main__":
    main()

Generated passphrase: 9V*WqD9wznH*Lqu!z&D4IY3AGzY7bWhS
Original message: short
Decrypted message: short


Embedding for 16794_Harpy_V1

In [23]:
def main():
    # Initialize steganography system
    stego = SecureFaceSteganography()

    # Load 3D face model (example path)
    face_model = stego.load_face_model('/content/drive/MyDrive/OutputPaper2/16794_Harpy_V1.obj')
    original_model = {key: value.copy() for key, value in face_model.items()}

    # Generate secure passphrase
    passphrase = stego.generate_passphrase()
    print(f"Generated passphrase: {passphrase}")

    # Message to hide - Reduced length to fit within the model's capacity
    secret_message = "short"

    # Encrypt the message
    encrypted_data = stego.encrypt_message(secret_message, passphrase)

    # Embed encrypted data in face model
    modified_model = stego.embed_in_face_model(face_model, encrypted_data)

    # Extract and decrypt
    extracted_data = stego.extract_from_face_model(
        modified_model, original_model, len(encrypted_data)
    )
    decrypted_message = stego.decrypt_message(extracted_data, passphrase)

    print(f"Original message: {secret_message}")
    print(f"Decrypted message: {decrypted_message}")

    # Save modified model
    mesh = trimesh.Trimesh(
        vertices=modified_model['vertices'],
        faces=modified_model['faces']
    )
    mesh.export('/content/drive/MyDrive/OutputPaper2/modified_16794_Harpy_V1.obj')

if __name__ == "__main__":
    main()

Generated passphrase: BOF7eU5RoaSkGM1#*GkEZuY#imOTaJyl
Original message: short
Decrypted message: short


Evaluation for 15800_Mummy_Hand_v1_NEW

In [20]:
#!pip install pyglet
#!pip install pyglet<2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import trimesh
from scipy.stats import wasserstein_distance
from skimage.metrics import structural_similarity
import math
from scipy.spatial import KDTree
import os

class Stego3DEvaluator:
    def __init__(self):
        """Initialize the 3D steganography evaluator"""
        self.metrics = {}

    def preprocess_models(self, original_mesh, stego_mesh):
        """Preprocess and align models to ensure compatibility"""
        # Center both meshes
        original_mesh.vertices -= original_mesh.vertices.mean(axis=0)
        stego_mesh.vertices -= stego_mesh.vertices.mean(axis=0)

        # Scale to unit cube
        original_scale = np.max(np.abs(original_mesh.vertices))
        stego_scale = np.max(np.abs(stego_mesh.vertices))

        original_mesh.vertices /= original_scale
        stego_mesh.vertices /= stego_scale

        return original_mesh, stego_mesh

    def align_vertices(self, original_vertices, stego_vertices):
        """Align vertices between models using nearest neighbor matching"""
        # Build KD-tree for faster nearest neighbor search
        tree = KDTree(original_vertices)

        # Find nearest neighbors for each stego vertex
        distances, indices = tree.query(stego_vertices)

        # Create aligned vertex arrays
        aligned_original = original_vertices[indices]
        aligned_stego = stego_vertices

        return aligned_original, aligned_stego, distances

    def load_models(self, original_path, stego_path, align=True):
        """Load and preprocess original and stego 3D models"""
        try:
            # Load meshes
            original_mesh = trimesh.load(original_path)
            stego_mesh = trimesh.load(stego_path)

            # Check if loaded data is a Scene and extract the first mesh if it is
            if isinstance(original_mesh, trimesh.Scene):
                original_mesh = next(iter(original_mesh.geometry.values())) # Get the first mesh from the scene
            if isinstance(stego_mesh, trimesh.Scene):
                stego_mesh = next(iter(stego_mesh.geometry.values())) # Get the first mesh from the scene

            print(f"Original vertices: {len(original_mesh.vertices)}")
            print(f"Stego vertices: {len(stego_mesh.vertices)}")

            # Preprocess meshes
            original_mesh, stego_mesh = self.preprocess_models(original_mesh, stego_mesh)

            if align:
                # Align vertices
                self.original_vertices, self.stego_vertices, distances = self.align_vertices(
                    original_mesh.vertices,
                    stego_mesh.vertices
                )

                # Print alignment statistics
                print(f"\nAlignment Statistics:")
                print(f"Mean distance: {np.mean(distances):.6f}")
                print(f"Max distance: {np.max(distances):.6f}")
                print(f"Aligned vertices: {len(self.original_vertices)}")
            else:
                if len(original_mesh.vertices) != len(stego_mesh.vertices):
                    raise ValueError("Models have different number of vertices and alignment is disabled")
                self.original_vertices = original_mesh.vertices
                self.stego_vertices = stego_mesh.vertices

            self.original_mesh = original_mesh
            self.stego_mesh = stego_mesh

            return True

        except Exception as e:
            print(f"Error loading models: {str(e)}")
            return False

    def calculate_vertex_error_map(self):
        """Calculate and visualize vertex-wise errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)
        return errors



    def calculate_psnr(self):
        """Calculate Peak Signal-to-Noise Ratio"""
        mse = np.mean((self.original_vertices - self.stego_vertices) ** 2)
        if mse == 0:
            return float('inf')

        max_val = np.max(self.original_vertices) - np.min(self.original_vertices)
        psnr = 20 * math.log10(max_val / math.sqrt(mse))
        self.metrics['PSNR'] = psnr
        return psnr

    def calculate_ssim(self):
        """Calculate Structural Similarity Index"""
        orig_reshaped = self.original_vertices.reshape(-1, 3)
        stego_reshaped = self.stego_vertices.reshape(-1, 3)

        ssim_scores = []
        for i in range(3):
            score = structural_similarity(
                orig_reshaped[:, i],
                stego_reshaped[:, i],
                data_range=np.max(orig_reshaped[:, i]) - np.min(orig_reshaped[:, i])
            )
            ssim_scores.append(score)

        ssim = np.mean(ssim_scores)
        self.metrics['SSIM'] = ssim
        return ssim

    def calculate_mse(self):
        """Calculate Mean Squared Error"""
        mse = mean_squared_error(self.original_vertices, self.stego_vertices)
        self.metrics['MSE'] = mse
        return mse

    def calculate_rmse(self):
        """Calculate Root Mean Squared Error"""
        rmse = np.sqrt(self.calculate_mse())
        self.metrics['RMSE'] = rmse
        return rmse

    def calculate_ber(self, threshold=1e-6):
        """Calculate Bit Error Rate"""
        differences = np.abs(self.original_vertices - self.stego_vertices)
        binary_orig = (self.original_vertices > threshold).astype(int)
        binary_stego = (self.stego_vertices > threshold).astype(int)

        total_bits = np.prod(binary_orig.shape)
        error_bits = np.sum(binary_orig != binary_stego)

        ber = error_bits / total_bits
        self.metrics['BER'] = ber
        return ber

    def calculate_hausdorff_distance(self):
        """Calculate Hausdorff distance between original and stego models"""
        def directed_hausdorff(source, target):
            tree = KDTree(target)
            distances, _ = tree.query(source)
            return np.max(distances)

        forward = directed_hausdorff(self.original_vertices, self.stego_vertices)
        backward = directed_hausdorff(self.stego_vertices, self.original_vertices)

        hausdorff = max(forward, backward)
        self.metrics['Hausdorff'] = hausdorff
        return hausdorff

    def calculate_histogram_similarity(self, bins=50):
        """Calculate histogram similarity using Wasserstein distance"""
        distances = []

        for i in range(3):
            hist_orig, _ = np.histogram(self.original_vertices[:, i], bins=bins, density=True)
            hist_stego, _ = np.histogram(self.stego_vertices[:, i], bins=bins, density=True)

            distance = wasserstein_distance(hist_orig, hist_stego)
            distances.append(distance)

        hist_similarity = 1 / (1 + np.mean(distances))
        self.metrics['Histogram_Similarity'] = hist_similarity
        return hist_similarity

    def plot_histograms(self, save_path=None):
        """Plot histograms of original and stego models"""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        coords = ['X', 'Y', 'Z']

        for i, (ax, coord) in enumerate(zip(axes, coords)):
            ax.hist(self.original_vertices[:, i], bins=50, alpha=0.5, label='Original', density=True)
            ax.hist(self.stego_vertices[:, i], bins=50, alpha=0.5, label='Stego', density=True)
            ax.set_title(f'{coord} Coordinate Distribution')
            ax.set_xlabel(f'{coord} Value')
            ax.set_ylabel('Density')
            ax.legend()

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def plot_error_distribution(self, save_path=None):
        """Plot the distribution of geometric errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)

        plt.figure(figsize=(10, 6))
        plt.hist(errors, bins=50, density=True)
        plt.title('Geometric Error Distribution')
        plt.xlabel('Error Magnitude')
        plt.ylabel('Density')

        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def evaluate_all(self, plot=True, save_plots=False):
        """Calculate all metrics and optionally generate plots"""
        metrics = {
            'PSNR': self.calculate_psnr(),
            'SSIM': self.calculate_ssim(),
            'MSE': self.calculate_mse(),
            'RMSE': self.calculate_rmse(),
            'BER': self.calculate_ber(),
            'Hausdorff': self.calculate_hausdorff_distance(),
            'Histogram_Similarity': self.calculate_histogram_similarity()
        }

        if plot:
            self.plot_histograms(save_path='/content/drive/MyDrive/OutputPaper2/histograms_15800_Mummy_Hand_v1_NEW_.png' if save_plots else None)
            self.plot_error_distribution(save_path='/content/drive/MyDrive/OutputPaper2/error_distribution_15800_Mummy_Hand_v1_NEW.png' if save_plots else None)
            #self.visualize_error_map(save_path='error_map.png' if save_plots else None)

        return metrics
    def generate_report(self, output_path='/content/drive/MyDrive/OutputPaper2/evaluation_report_15800_Mummy_Hand_v1_NEW.txt'):
        """Generate a detailed evaluation report"""
        with open(output_path, 'w') as f:
            f.write("3D Steganography Evaluation Report\n")
            f.write("=================================\n\n")

            # Model information
            f.write("Model Information:\n")
            f.write(f"Number of vertices: {len(self.original_vertices)}\n")
            f.write(f"Number of faces: {len(self.original_mesh.faces)}\n\n")

            # Metrics
            f.write("Quality Metrics:\n")
            for metric, value in self.metrics.items():
                f.write(f"{metric}: {value:.6f}\n")

            f.write("\nInterpretation:\n")
            f.write("- PSNR > 30 dB typically indicates good quality\n")
            f.write("- SSIM closer to 1 indicates better structural preservation\n")
            f.write("- Lower MSE and RMSE values indicate better similarity\n")
            f.write("- Lower BER indicates better steganographic accuracy\n")
            f.write("- Histogram similarity closer to 1 indicates better statistical imperceptibility\n")

def main():
    # Initialize evaluator
    evaluator = Stego3DEvaluator()

    # Load and align models
    if not evaluator.load_models('/content/drive/MyDrive/OutputPaper2/modified_15800_Mummy_Hand_v1_NEW.obj', '/content/drive/MyDrive/OutputPaper2/15800_Mummy_Hand_v1_NEW.obj', align=True):
        return

    # Perform evaluation
    metrics = evaluator.evaluate_all(plot=True, save_plots=True)

    # Print results
    print("\nEvaluation Results:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.6f}")
    evaluator.generate_report()
    print("\nDetailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report_15800_Mummy_Hand_v1_NEW.txt'")

if __name__ == "__main__":
    main()

Original vertices: 48998
Stego vertices: 49098

Alignment Statistics:
Mean distance: 0.001654
Max distance: 0.003119
Aligned vertices: 49098

Evaluation Results:
PSNR: 64.761729
SSIM: 0.999883
MSE: 0.000001
RMSE: 0.001051
BER: 0.000401
Hausdorff: 0.003119
Histogram_Similarity: 0.996925

Detailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report_15800_Mummy_Hand_v1_NEW.txt'


Evaluation for 15742_Zombie_Arm_v1

In [22]:
#!pip install pyglet
#!pip install pyglet<2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import trimesh
from scipy.stats import wasserstein_distance
from skimage.metrics import structural_similarity
import math
from scipy.spatial import KDTree
import os

class Stego3DEvaluator:
    def __init__(self):
        """Initialize the 3D steganography evaluator"""
        self.metrics = {}

    def preprocess_models(self, original_mesh, stego_mesh):
        """Preprocess and align models to ensure compatibility"""
        # Center both meshes
        original_mesh.vertices -= original_mesh.vertices.mean(axis=0)
        stego_mesh.vertices -= stego_mesh.vertices.mean(axis=0)

        # Scale to unit cube
        original_scale = np.max(np.abs(original_mesh.vertices))
        stego_scale = np.max(np.abs(stego_mesh.vertices))

        original_mesh.vertices /= original_scale
        stego_mesh.vertices /= stego_scale

        return original_mesh, stego_mesh

    def align_vertices(self, original_vertices, stego_vertices):
        """Align vertices between models using nearest neighbor matching"""
        # Build KD-tree for faster nearest neighbor search
        tree = KDTree(original_vertices)

        # Find nearest neighbors for each stego vertex
        distances, indices = tree.query(stego_vertices)

        # Create aligned vertex arrays
        aligned_original = original_vertices[indices]
        aligned_stego = stego_vertices

        return aligned_original, aligned_stego, distances

    def load_models(self, original_path, stego_path, align=True):
        """Load and preprocess original and stego 3D models"""
        try:
            # Load meshes
            original_mesh = trimesh.load(original_path)
            stego_mesh = trimesh.load(stego_path)

            # Check if loaded data is a Scene and extract the first mesh if it is
            if isinstance(original_mesh, trimesh.Scene):
                original_mesh = next(iter(original_mesh.geometry.values())) # Get the first mesh from the scene
            if isinstance(stego_mesh, trimesh.Scene):
                stego_mesh = next(iter(stego_mesh.geometry.values())) # Get the first mesh from the scene

            print(f"Original vertices: {len(original_mesh.vertices)}")
            print(f"Stego vertices: {len(stego_mesh.vertices)}")

            # Preprocess meshes
            original_mesh, stego_mesh = self.preprocess_models(original_mesh, stego_mesh)

            if align:
                # Align vertices
                self.original_vertices, self.stego_vertices, distances = self.align_vertices(
                    original_mesh.vertices,
                    stego_mesh.vertices
                )

                # Print alignment statistics
                print(f"\nAlignment Statistics:")
                print(f"Mean distance: {np.mean(distances):.6f}")
                print(f"Max distance: {np.max(distances):.6f}")
                print(f"Aligned vertices: {len(self.original_vertices)}")
            else:
                if len(original_mesh.vertices) != len(stego_mesh.vertices):
                    raise ValueError("Models have different number of vertices and alignment is disabled")
                self.original_vertices = original_mesh.vertices
                self.stego_vertices = stego_mesh.vertices

            self.original_mesh = original_mesh
            self.stego_mesh = stego_mesh

            return True

        except Exception as e:
            print(f"Error loading models: {str(e)}")
            return False

    def calculate_vertex_error_map(self):
        """Calculate and visualize vertex-wise errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)
        return errors



    def calculate_psnr(self):
        """Calculate Peak Signal-to-Noise Ratio"""
        mse = np.mean((self.original_vertices - self.stego_vertices) ** 2)
        if mse == 0:
            return float('inf')

        max_val = np.max(self.original_vertices) - np.min(self.original_vertices)
        psnr = 20 * math.log10(max_val / math.sqrt(mse))
        self.metrics['PSNR'] = psnr
        return psnr

    def calculate_ssim(self):
        """Calculate Structural Similarity Index"""
        orig_reshaped = self.original_vertices.reshape(-1, 3)
        stego_reshaped = self.stego_vertices.reshape(-1, 3)

        ssim_scores = []
        for i in range(3):
            score = structural_similarity(
                orig_reshaped[:, i],
                stego_reshaped[:, i],
                data_range=np.max(orig_reshaped[:, i]) - np.min(orig_reshaped[:, i])
            )
            ssim_scores.append(score)

        ssim = np.mean(ssim_scores)
        self.metrics['SSIM'] = ssim
        return ssim

    def calculate_mse(self):
        """Calculate Mean Squared Error"""
        mse = mean_squared_error(self.original_vertices, self.stego_vertices)
        self.metrics['MSE'] = mse
        return mse

    def calculate_rmse(self):
        """Calculate Root Mean Squared Error"""
        rmse = np.sqrt(self.calculate_mse())
        self.metrics['RMSE'] = rmse
        return rmse

    def calculate_ber(self, threshold=1e-6):
        """Calculate Bit Error Rate"""
        differences = np.abs(self.original_vertices - self.stego_vertices)
        binary_orig = (self.original_vertices > threshold).astype(int)
        binary_stego = (self.stego_vertices > threshold).astype(int)

        total_bits = np.prod(binary_orig.shape)
        error_bits = np.sum(binary_orig != binary_stego)

        ber = error_bits / total_bits
        self.metrics['BER'] = ber
        return ber

    def calculate_hausdorff_distance(self):
        """Calculate Hausdorff distance between original and stego models"""
        def directed_hausdorff(source, target):
            tree = KDTree(target)
            distances, _ = tree.query(source)
            return np.max(distances)

        forward = directed_hausdorff(self.original_vertices, self.stego_vertices)
        backward = directed_hausdorff(self.stego_vertices, self.original_vertices)

        hausdorff = max(forward, backward)
        self.metrics['Hausdorff'] = hausdorff
        return hausdorff

    def calculate_histogram_similarity(self, bins=50):
        """Calculate histogram similarity using Wasserstein distance"""
        distances = []

        for i in range(3):
            hist_orig, _ = np.histogram(self.original_vertices[:, i], bins=bins, density=True)
            hist_stego, _ = np.histogram(self.stego_vertices[:, i], bins=bins, density=True)

            distance = wasserstein_distance(hist_orig, hist_stego)
            distances.append(distance)

        hist_similarity = 1 / (1 + np.mean(distances))
        self.metrics['Histogram_Similarity'] = hist_similarity
        return hist_similarity

    def plot_histograms(self, save_path=None):
        """Plot histograms of original and stego models"""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        coords = ['X', 'Y', 'Z']

        for i, (ax, coord) in enumerate(zip(axes, coords)):
            ax.hist(self.original_vertices[:, i], bins=50, alpha=0.5, label='Original', density=True)
            ax.hist(self.stego_vertices[:, i], bins=50, alpha=0.5, label='Stego', density=True)
            ax.set_title(f'{coord} Coordinate Distribution')
            ax.set_xlabel(f'{coord} Value')
            ax.set_ylabel('Density')
            ax.legend()

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def plot_error_distribution(self, save_path=None):
        """Plot the distribution of geometric errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)

        plt.figure(figsize=(10, 6))
        plt.hist(errors, bins=50, density=True)
        plt.title('Geometric Error Distribution')
        plt.xlabel('Error Magnitude')
        plt.ylabel('Density')

        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def evaluate_all(self, plot=True, save_plots=False):
        """Calculate all metrics and optionally generate plots"""
        metrics = {
            'PSNR': self.calculate_psnr(),
            'SSIM': self.calculate_ssim(),
            'MSE': self.calculate_mse(),
            'RMSE': self.calculate_rmse(),
            'BER': self.calculate_ber(),
            'Hausdorff': self.calculate_hausdorff_distance(),
            'Histogram_Similarity': self.calculate_histogram_similarity()
        }

        if plot:
            self.plot_histograms(save_path='/content/drive/MyDrive/OutputPaper2/histograms_15742_Zombie_Arm_v1_.png' if save_plots else None)
            self.plot_error_distribution(save_path='/content/drive/MyDrive/OutputPaper2/error_distribution_15742_Zombie_Arm_v1.png' if save_plots else None)
            #self.visualize_error_map(save_path='error_map.png' if save_plots else None)

        return metrics
    def generate_report(self, output_path='/content/drive/MyDrive/OutputPaper2/evaluation_report_15742_Zombie_Arm_v1.txt'):
        """Generate a detailed evaluation report"""
        with open(output_path, 'w') as f:
            f.write("3D Steganography Evaluation Report\n")
            f.write("=================================\n\n")

            # Model information
            f.write("Model Information:\n")
            f.write(f"Number of vertices: {len(self.original_vertices)}\n")
            f.write(f"Number of faces: {len(self.original_mesh.faces)}\n\n")

            # Metrics
            f.write("Quality Metrics:\n")
            for metric, value in self.metrics.items():
                f.write(f"{metric}: {value:.6f}\n")

            f.write("\nInterpretation:\n")
            f.write("- PSNR > 30 dB typically indicates good quality\n")
            f.write("- SSIM closer to 1 indicates better structural preservation\n")
            f.write("- Lower MSE and RMSE values indicate better similarity\n")
            f.write("- Lower BER indicates better steganographic accuracy\n")
            f.write("- Histogram similarity closer to 1 indicates better statistical imperceptibility\n")

def main():
    # Initialize evaluator
    evaluator = Stego3DEvaluator()

    # Load and align models
    if not evaluator.load_models('/content/drive/MyDrive/OutputPaper2/modified_15742_Zombie_Arm_v1.obj', '/content/drive/MyDrive/OutputPaper2/15742_Zombie_Arm_v1.obj', align=True):
        return

    # Perform evaluation
    metrics = evaluator.evaluate_all(plot=True, save_plots=True)

    # Print results
    print("\nEvaluation Results:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.6f}")
    evaluator.generate_report()
    print("\nDetailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report_15742_Zombie_Arm_v1.txt'")

if __name__ == "__main__":
    main()

Original vertices: 37385
Stego vertices: 37472

Alignment Statistics:
Mean distance: 0.002288
Max distance: 0.003917
Aligned vertices: 37472

Evaluation Results:
PSNR: 61.106745
SSIM: 0.999591
MSE: 0.000002
RMSE: 0.001428
BER: 0.000845
Hausdorff: 0.003917
Histogram_Similarity: 0.992270

Detailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report_15742_Zombie_Arm_v1.txt'


Evaluation for 16794_Harpy_V1

In [24]:
#!pip install pyglet
#!pip install pyglet<2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import trimesh
from scipy.stats import wasserstein_distance
from skimage.metrics import structural_similarity
import math
from scipy.spatial import KDTree
import os

class Stego3DEvaluator:
    def __init__(self):
        """Initialize the 3D steganography evaluator"""
        self.metrics = {}

    def preprocess_models(self, original_mesh, stego_mesh):
        """Preprocess and align models to ensure compatibility"""
        # Center both meshes
        original_mesh.vertices -= original_mesh.vertices.mean(axis=0)
        stego_mesh.vertices -= stego_mesh.vertices.mean(axis=0)

        # Scale to unit cube
        original_scale = np.max(np.abs(original_mesh.vertices))
        stego_scale = np.max(np.abs(stego_mesh.vertices))

        original_mesh.vertices /= original_scale
        stego_mesh.vertices /= stego_scale

        return original_mesh, stego_mesh

    def align_vertices(self, original_vertices, stego_vertices):
        """Align vertices between models using nearest neighbor matching"""
        # Build KD-tree for faster nearest neighbor search
        tree = KDTree(original_vertices)

        # Find nearest neighbors for each stego vertex
        distances, indices = tree.query(stego_vertices)

        # Create aligned vertex arrays
        aligned_original = original_vertices[indices]
        aligned_stego = stego_vertices

        return aligned_original, aligned_stego, distances

    def load_models(self, original_path, stego_path, align=True):
        """Load and preprocess original and stego 3D models"""
        try:
            # Load meshes
            original_mesh = trimesh.load(original_path)
            stego_mesh = trimesh.load(stego_path)

            # Check if loaded data is a Scene and extract the first mesh if it is
            if isinstance(original_mesh, trimesh.Scene):
                original_mesh = next(iter(original_mesh.geometry.values())) # Get the first mesh from the scene
            if isinstance(stego_mesh, trimesh.Scene):
                stego_mesh = next(iter(stego_mesh.geometry.values())) # Get the first mesh from the scene

            print(f"Original vertices: {len(original_mesh.vertices)}")
            print(f"Stego vertices: {len(stego_mesh.vertices)}")

            # Preprocess meshes
            original_mesh, stego_mesh = self.preprocess_models(original_mesh, stego_mesh)

            if align:
                # Align vertices
                self.original_vertices, self.stego_vertices, distances = self.align_vertices(
                    original_mesh.vertices,
                    stego_mesh.vertices
                )

                # Print alignment statistics
                print(f"\nAlignment Statistics:")
                print(f"Mean distance: {np.mean(distances):.6f}")
                print(f"Max distance: {np.max(distances):.6f}")
                print(f"Aligned vertices: {len(self.original_vertices)}")
            else:
                if len(original_mesh.vertices) != len(stego_mesh.vertices):
                    raise ValueError("Models have different number of vertices and alignment is disabled")
                self.original_vertices = original_mesh.vertices
                self.stego_vertices = stego_mesh.vertices

            self.original_mesh = original_mesh
            self.stego_mesh = stego_mesh

            return True

        except Exception as e:
            print(f"Error loading models: {str(e)}")
            return False

    def calculate_vertex_error_map(self):
        """Calculate and visualize vertex-wise errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)
        return errors



    def calculate_psnr(self):
        """Calculate Peak Signal-to-Noise Ratio"""
        mse = np.mean((self.original_vertices - self.stego_vertices) ** 2)
        if mse == 0:
            return float('inf')

        max_val = np.max(self.original_vertices) - np.min(self.original_vertices)
        psnr = 20 * math.log10(max_val / math.sqrt(mse))
        self.metrics['PSNR'] = psnr
        return psnr

    def calculate_ssim(self):
        """Calculate Structural Similarity Index"""
        orig_reshaped = self.original_vertices.reshape(-1, 3)
        stego_reshaped = self.stego_vertices.reshape(-1, 3)

        ssim_scores = []
        for i in range(3):
            score = structural_similarity(
                orig_reshaped[:, i],
                stego_reshaped[:, i],
                data_range=np.max(orig_reshaped[:, i]) - np.min(orig_reshaped[:, i])
            )
            ssim_scores.append(score)

        ssim = np.mean(ssim_scores)
        self.metrics['SSIM'] = ssim
        return ssim

    def calculate_mse(self):
        """Calculate Mean Squared Error"""
        mse = mean_squared_error(self.original_vertices, self.stego_vertices)
        self.metrics['MSE'] = mse
        return mse

    def calculate_rmse(self):
        """Calculate Root Mean Squared Error"""
        rmse = np.sqrt(self.calculate_mse())
        self.metrics['RMSE'] = rmse
        return rmse

    def calculate_ber(self, threshold=1e-6):
        """Calculate Bit Error Rate"""
        differences = np.abs(self.original_vertices - self.stego_vertices)
        binary_orig = (self.original_vertices > threshold).astype(int)
        binary_stego = (self.stego_vertices > threshold).astype(int)

        total_bits = np.prod(binary_orig.shape)
        error_bits = np.sum(binary_orig != binary_stego)

        ber = error_bits / total_bits
        self.metrics['BER'] = ber
        return ber

    def calculate_hausdorff_distance(self):
        """Calculate Hausdorff distance between original and stego models"""
        def directed_hausdorff(source, target):
            tree = KDTree(target)
            distances, _ = tree.query(source)
            return np.max(distances)

        forward = directed_hausdorff(self.original_vertices, self.stego_vertices)
        backward = directed_hausdorff(self.stego_vertices, self.original_vertices)

        hausdorff = max(forward, backward)
        self.metrics['Hausdorff'] = hausdorff
        return hausdorff

    def calculate_histogram_similarity(self, bins=50):
        """Calculate histogram similarity using Wasserstein distance"""
        distances = []

        for i in range(3):
            hist_orig, _ = np.histogram(self.original_vertices[:, i], bins=bins, density=True)
            hist_stego, _ = np.histogram(self.stego_vertices[:, i], bins=bins, density=True)

            distance = wasserstein_distance(hist_orig, hist_stego)
            distances.append(distance)

        hist_similarity = 1 / (1 + np.mean(distances))
        self.metrics['Histogram_Similarity'] = hist_similarity
        return hist_similarity

    def plot_histograms(self, save_path=None):
        """Plot histograms of original and stego models"""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        coords = ['X', 'Y', 'Z']

        for i, (ax, coord) in enumerate(zip(axes, coords)):
            ax.hist(self.original_vertices[:, i], bins=50, alpha=0.5, label='Original', density=True)
            ax.hist(self.stego_vertices[:, i], bins=50, alpha=0.5, label='Stego', density=True)
            ax.set_title(f'{coord} Coordinate Distribution')
            ax.set_xlabel(f'{coord} Value')
            ax.set_ylabel('Density')
            ax.legend()

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def plot_error_distribution(self, save_path=None):
        """Plot the distribution of geometric errors"""
        errors = np.linalg.norm(self.original_vertices - self.stego_vertices, axis=1)

        plt.figure(figsize=(10, 6))
        plt.hist(errors, bins=50, density=True)
        plt.title('Geometric Error Distribution')
        plt.xlabel('Error Magnitude')
        plt.ylabel('Density')

        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    def evaluate_all(self, plot=True, save_plots=False):
        """Calculate all metrics and optionally generate plots"""
        metrics = {
            'PSNR': self.calculate_psnr(),
            'SSIM': self.calculate_ssim(),
            'MSE': self.calculate_mse(),
            'RMSE': self.calculate_rmse(),
            'BER': self.calculate_ber(),
            'Hausdorff': self.calculate_hausdorff_distance(),
            'Histogram_Similarity': self.calculate_histogram_similarity()
        }

        if plot:
            self.plot_histograms(save_path='/content/drive/MyDrive/OutputPaper2/histograms_16794_Harpy_V1_.png' if save_plots else None)
            self.plot_error_distribution(save_path='/content/drive/MyDrive/OutputPaper2/error_distribution_16794_Harpy_V1.png' if save_plots else None)
            #self.visualize_error_map(save_path='error_map.png' if save_plots else None)

        return metrics
    def generate_report(self, output_path='/content/drive/MyDrive/OutputPaper2/evaluation_report_16794_Harpy_V1.txt'):
        """Generate a detailed evaluation report"""
        with open(output_path, 'w') as f:
            f.write("3D Steganography Evaluation Report\n")
            f.write("=================================\n\n")

            # Model information
            f.write("Model Information:\n")
            f.write(f"Number of vertices: {len(self.original_vertices)}\n")
            f.write(f"Number of faces: {len(self.original_mesh.faces)}\n\n")

            # Metrics
            f.write("Quality Metrics:\n")
            for metric, value in self.metrics.items():
                f.write(f"{metric}: {value:.6f}\n")

            f.write("\nInterpretation:\n")
            f.write("- PSNR > 30 dB typically indicates good quality\n")
            f.write("- SSIM closer to 1 indicates better structural preservation\n")
            f.write("- Lower MSE and RMSE values indicate better similarity\n")
            f.write("- Lower BER indicates better steganographic accuracy\n")
            f.write("- Histogram similarity closer to 1 indicates better statistical imperceptibility\n")

def main():
    # Initialize evaluator
    evaluator = Stego3DEvaluator()

    # Load and align models
    if not evaluator.load_models('/content/drive/MyDrive/OutputPaper2/modified_16794_Harpy_V1.obj', '/content/drive/MyDrive/OutputPaper2/16794_Harpy_V1.obj', align=True):
        return

    # Perform evaluation
    metrics = evaluator.evaluate_all(plot=True, save_plots=True)

    # Print results
    print("\nEvaluation Results:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.6f}")
    evaluator.generate_report()
    print("\nDetailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report_16794_Harpy_V1.txt'")

if __name__ == "__main__":
    main()

Original vertices: 65981
Stego vertices: 66143

Alignment Statistics:
Mean distance: 0.001147
Max distance: 0.001282
Aligned vertices: 66143

Evaluation Results:
PSNR: 69.562818
SSIM: 0.999897
MSE: 0.000000
RMSE: 0.000665
BER: 0.000297
Hausdorff: 0.001282
Histogram_Similarity: 0.998757

Detailed report saved to '/content/drive/MyDrive/OutputPaper2/evaluation_report_16794_Harpy_V1.txt'
